# PLANNING GRAPH PART

In [2]:
#%%

%load_ext autoreload
%autoreload 2

import numpy as np
import numpy.random as rd
from scipy import stats
from planning_graph.planning_graph import PlanningGraph, NoOpAction
from pddlpy.pddl import Operator
from typing import Set, Tuple, List

In [13]:
from planning_graph.planning_graph import PlanningGraph
from planning_graph.planning_graph_planner import GraphPlanner


planning_graph = PlanningGraph("domain/gripper/domain.pddl", 
                               'domain/gripper/train/gripper-n3.pddl')
graph = planning_graph.create(max_num_of_levels=100)
print("finish graph")
goal = planning_graph.goal
graph_planner = GraphPlanner()
layered_plan = graph_planner.plan(graph, goal)
print(layered_plan)
t = 0
for i in range(len(layered_plan._layered_plan)):
    print(i)
    for a in (layered_plan[i]._plan):
        if not isinstance(a, NoOpAction):
            print(a.operator_name)
            t+=1
            
print(t)

finish graph


TypeError: plan() missing 1 required positional argument: 'pg'

In [5]:
def generate_feature_vec(planning_graph, state, max_level, visual = False, title = None):
    """
    Generate the feature vector followed by the paper from the input (problem, state) pair as described by the paper
    
    Inputs
    ----------
    planning_graph: the problem pi
    state: the current state s
    max_level: the maximum layer allowed for ff algorithm to do forward expanding
    visual/title: whether need to generate a photo representing the graph, default False/None, 
        if need to generate a photo with name as title, visual must be true and title must not be none
    
    Outputs
    ----------
    feature_vec: a vector of length n + 2*n**2 + 3 representing the feature generated from the given (problem, state) pair
                 the first n values are single action feature
                 the second 2*n**2 are pairwise action feature
                 the last 3 are original heuristic value, the number of layers in pi and the number of unsatisfied goals
    """
    # draw graph first
    graph = planning_graph.create_with_state(state, max_num_of_levels=max_level, visualize = visual)
    if title is not None and visual:
        graph.visualize_png(title)
    
    # get action schema and output list
    act_schema = np.array(list(planning_graph._planning_problem._domain_problem.operators())) # store names of total action schema
    n = len(act_schema)  # length of action schema
    feature_vec = np.zeros(n+2*n**2+3) # return feature vec, first n is linear, second 2*n**2 is pairwise, last 3 is additional feature
    act_layers = list(graph.act.values()) # list of layers generated, ith value is the list of actions connencting i-1 th states to ith states layer

    # extract linear feature
    #-------------------------------------
    # ith value indicate the num of occurance for ith action of act_schema in the entire graph 
    counter = np.zeros(n)
    for act_layer in act_layers:
        if act_layer is not None:
            for a in act_layer:
                 if not isinstance(a, NoOpAction):
                        counter[act_schema == a.operator_name] += 1
    feature_vec[0:n] = counter
    
    # extract pair-wise feature
    #-------------------------------------
    # each pair a1, a2 is stored in n + [2*(n*a1+a2), 2*(n*a1+a2)+1]
    # e.g. when a1 is 1, a2 is 3, n is 5, store in 5 + [2*(8),  2*(8)+1]
    def to_index(n, index_a1, index_a2, adder):
        """
        return corresponding index in the position of the feature vector
        adder is either 0 or 1
        index_a1, index_a2 refer to move index in act_schema
        """
        return n+2*(n*index_a1+index_a2)+adder
    
    
    def append_to_dict(a, pre, eff_pos):
        """
        add action a into the dicitonary pre and eff_pos
        """
        for p in a.precondition_pos:
            current = pre.get(p)
            if current is None:
                current = [a]
            else:
                current.append(a)
            pre[p] = current
            
        for p in a.effect_pos:
            current = eff_pos.get(p)
            if current is None:
                current = [a]
            else:
                current.append(a)
            eff_pos[p] = current
            
#         for p in a.effect_neg:
#             current = effect_neg.get(p)
#             if current is None:
#                 current = [name]
#             else:
#                 current.append(name)
#             effect_neg[p] = current
#         return pre, eff_pos, eff_neg
            return pre, eff_pos


    # define dictionary variables for comparison purpose
    pre = {}
    eff_pos = {}
#     eff_neg = {}
    
    # add pre and eff into the empty dictionary for the first layer
    for a in act_layers[1]:
        if not isinstance(a, NoOpAction):
            pre, eff_pos = append_to_dict(a, pre, eff_pos)
   
    # loop through second to last action layers
    for i in range(2,len(act_layers)): 
        act_layer = act_layers[i]
        
        # update fecture vec for the entire layer
        for a2 in act_layer:
            if not isinstance(a2, NoOpAction):
                # count for num of occurances, use set to avoid multiple countsc
                s1 = set() # feature 1 where eff a1 and pre a2 has intersections
                s2 = set() # feature 2 where pre a1 and eff a2 has intersections          
                for p in a2.precondition_pos:
                    current = eff_pos.get(p)
                    if current is not None:
                        for a1 in current:
                            s1.add(a1) 

                for p in a2.effect_pos:
                    current = pre.get(p)
                    if current is not None:
                        for a1 in current:
                            s2.add(a1)
                            
                # add index to feature_vec based on set generated:
                index_a2 = int(np.where(a2.operator_name == act_schema)[0])
                for a1 in s1:
                    # update feature 1 for pair (a1, a2)
                    index_a1 = int(np.where(a1.operator_name == act_schema)[0])
                    feature_vec[to_index(n, index_a1, index_a2,0)]+=1
      
                for a1 in s2:
                    # update feature 2 for pair (a1, a2)
                    index_a1 = int(np.where(a1.operator_name == act_schema)[0])
                    feature_vec[to_index(n, index_a1, index_a2,1)]+=1

        # update pre and eff_pos for the entire layer
        for a2 in act_layer:
            if not isinstance(a2, NoOpAction):
                pre, eff_pos = append_to_dict(a2, pre, eff_pos)
           
    # extract final features
    #-------------------------------------
    # add heuristic value, number of layers and number of unsatisfied goals
    goal = planning_graph.goal
    graph_planner = GraphPlanner()
    layered_plan = graph_planner.plan(graph, goal, planning_graph)
    total_len = len(feature_vec)
    # number of layers:
    feature_vec[total_len - 3] = len(act_layers)
    # heuristic value hFF: (number of total actions in the plan)
    h_v = 0
    for i in range(len(layered_plan._layered_plan)):
        for a in (layered_plan[i]._plan):
            if not isinstance(a, NoOpAction):
                h_v += 1
    feature_vec[total_len - 2] = h_v
    
    ### ISSUE: UNSATISFIED GOAL FEATURE
    #-------------------------------------
#     # unsatisfied goal (2 ** (last layer total pos num - goal state pos num))
#     last = graph.prop[len(graph.prop)-1]
#     feature_vec[total_len - 1] = 2 ** (len(last) - len(goal))
    #-------------------------------------
    

    return feature_vec
            
        

In [6]:
def find_operator(action : str, op_list: List[Operator]):
    """
    find an operator from the planning graph's ground operator lists
    
    return: the action operator if found
    """
    name_list = action.replace('(', '').replace(')', '').split(' ')
    for op in op_list:
        if op.operator_name == name_list[0]:
            if list(op.variable_list.values()) == name_list[1:]: return op
    return None

def apply_operation(action: Operator, state : Set[Tuple]):
    """
    apply an action onto the input state
    
    return: the new state
    """
    new_state = state.copy()
    for eff in action.effect_pos:
        new_state.add(eff)
    for eff in action.effect_neg:
        new_state.remove(eff)
    return new_state

def read_plan(plan_file_path: str):
    """
    read all the lines from a plan file directory, remove the last line containing cost
    
    return: a list containing the ground truth plan with length equal to total cost
    """
    with open(plan_file_path, "r") as f:

        # Read the lines of the file into a list of strings
        lines = [line.strip() for line in f.readlines()]

    return lines[:-1]


In [7]:
import copy

def generate_training_data(domain_file_path, task_file_path, plan_file_path, problem_num : int, visual = False):
    """
    generate the feature vector matrix X together with a cost vector y from the given input
    
    Inputs
    ----------
    domain_file_path: the input domain file
    task_file_path: the input problem file
    plan_file_path: the input log file that store the optimal plan
    problem_num: the problem index for this domain
    
    Returns
    ----------
    None, None if no plan can be found (plan has cost 0)
    X : array, shape (plan_length-1, n_features)
        The input feature vec of states from initial states all the way towards the second-last state (one state before goal state)
    Y : array, shape (plan_length-1, 2)
        The input cost vector. If it's a 2D array
        The first column is the true cost pi optimal (assume unit cost)
        The second column is the probelm_num representing the index of this problem
    
    """
    # generate plan and get max level
    plan_actions = read_plan(plan_file_path)
    if len(plan_actions) == 0:
        return None, None
    max_level = max(10, len(plan_actions)+1)
    max_level = len(plan_actions)+2
    
    # generate graph
    planning_graph = PlanningGraph(domain_file_path, task_file_path, visualize = visual)
    graph = planning_graph.create(max_num_of_levels = max_level)
    ground_operators = planning_graph._planning_problem._get_ground_operators()
    
    # define output matrixes
    X = []
    y = []
    
    # loop from the final plan
    current_state = planning_graph._planning_problem.initial_state
    current_cost = len(plan_actions)
    for i in range(0,len(plan_actions)-1):
        X.append(generate_feature_vec(planning_graph, current_state, max_level))
#         X.append(generate_feature_vec(planning_graph, current_state, max_level, visual = True, title = f"test_image{i}th layer.png"))
        y.append(current_cost)
        current_action = find_operator(plan_actions[i], ground_operators)
        current_state = apply_operation(current_action, current_state)
        current_cost -=1
        
    y = np.c_[y, problem_num * np.ones(len(y))]
    return np.asarray(X), np.asarray(y)

In [8]:
X, y = generate_training_data('domain/blocks/domain.pddl', 'domain/blocks/blocks/blocks3/task07.pddl', 'domain/blocks/plans/blocks3-task07_1800.out', 5)

In [9]:
print(X.shape, y.shape)
print(type(X), type(y))
print(X)
print(y)
print(type(y[0,0]))

(3, 39) (3, 2)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
[[ 5.  4.  7.  5.  0.  0.  2.  4.  3.  7.  0.  1.  4.  2.  0.  0.  1.  0.
   5.  3.  2.  3.  0.  2.  2.  4.  5.  5.  0.  0.  2.  4.  4.  7.  0.  1.
   4.  4.  0.]
 [ 6.  5. 10.  4.  0.  0.  3.  9.  6. 18.  0.  2.  2.  2.  0.  0.  2.  0.
   4.  4.  4.  4.  0.  2.  4.  4.  8.  8.  0.  0.  2.  6.  4. 12.  0.  0.
   4.  3.  0.]
 [ 6.  3.  6.  0.  0.  0.  3.  9.  6. 18.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   3.  2.  0.]]
[[4. 5.]
 [3. 5.]
 [2. 5.]]
<class 'numpy.float64'>


# GENERATING DATA PART

In [10]:
import os

def generate_problem_matrix(domain_file_path, problem_folder_path, log_folder_path, output_path, title):
    
    # get the training problem names
    problem_name_list = [f.split('.')[0] for f in os.listdir(problem_folder_path)]
#     problem_name_list = ["task"+ str(i) for i in range(10,50)]
    X = None
    Y = None
    problem_index = 0
    print(problem_name_list)
    
#     # filter the too complex problems
#     p_l = problem_name_list.copy()
#     problem_name_list = []
#     for p in p_l:
#         if int(p[6])>=3 and int(p[6]) <= 6:
#             problem_name_list.append(p)
#     print(problem_name_list)
    
    # generate the final vectors
    for prob in problem_name_list:
        problem_path = problem_folder_path + "/" + prob + ".pddl"
        plan_path = log_folder_path + "/" + prob + "_1800.out"
        print(problem_path, plan_path)
        try:
            temp_X, temp_Y = generate_training_data(domain_file_path, problem_path, plan_path, problem_index)
            if X is None:
                X = temp_X
                Y = temp_Y
            elif temp_X is not None:
                X = np.vstack((X, temp_X))
                Y = np.vstack((Y, temp_Y))
                print(problem_path,X.shape, Y.shape)
            problem_index += 1
        except:
            continue
        
        
    # save the final training vectors
    np.savez(output_path+"/"+title+".npz", feature = X, label = Y)
        
        
    
    

In [10]:
def generate_problem_matrix(domain_file_path, problem_folder_path, log_folder_path, output_path, title):
    """
    Generate the corresponding feature/label matrix from the given inputs
    Stores in the format of "title.npz" in the output_path
    Each npz file contain two attribute: "feature" and "label"
    
    Inputs
    ----------
    domain_file_path: the path to the domain.pddl
    problem_folder_path: the path to the problem_folder containing all the task problem.pddl for generating vectors
    plan_folder_path: the plan folder that contains all the log files corresponding to each problem task
    output_path: the place to store the generated problem matrix
    title: the name for the output npz file
    """
    # get the training problem names and initialise parameters
    problem_name_list = [f.split('.')[0] for f in os.listdir(problem_folder_path)]
    X = None
    Y = None
    problem_index = 0
    
    # generate the final vectors in X, Y
    for prob in problem_name_list:
        problem_path = problem_folder_path + "/" + prob + ".pddl"
        plan_path = log_folder_path + "/" + prob + "_1800.out"
        try:
            temp_X, temp_Y = generate_training_data(domain_file_path, problem_path, plan_path, problem_index)
            if X is None:
                X = temp_X
                Y = temp_Y
            elif temp_X is not None:
                X = np.vstack((X, temp_X))
                Y = np.vstack((Y, temp_Y))
                print(problem_path,X.shape, Y.shape)
            problem_index += 1
        except:
            continue
        
        
    # save the final training vectors
    np.savez(output_path+"/"+title+".npz", feature = X, label = Y)
        
        
    
    

In [ ]:
generate_problem_matrix("domain/blocks/domain.pddl","domain/blocks/train-10","domain/blocks/plans","tests/test_vectors", "blocks")

['blocks10-task01', 'blocks10-task02', 'blocks10-task03', 'blocks10-task04', 'blocks10-task05', 'blocks10-task06', 'blocks10-task07', 'blocks10-task08', 'blocks10-task09', 'blocks10-task10', 'blocks3-task01', 'blocks3-task02', 'blocks3-task03', 'blocks3-task04', 'blocks3-task05', 'blocks3-task06', 'blocks3-task07', 'blocks3-task08', 'blocks3-task09', 'blocks3-task10', 'blocks4-task01', 'blocks4-task02', 'blocks4-task03', 'blocks4-task04', 'blocks4-task05', 'blocks4-task06', 'blocks4-task07', 'blocks4-task08', 'blocks4-task09', 'blocks4-task10', 'blocks5-task01', 'blocks5-task02', 'blocks5-task03', 'blocks5-task04', 'blocks5-task05', 'blocks5-task06', 'blocks5-task07', 'blocks5-task08', 'blocks5-task09', 'blocks5-task10', 'blocks6-task01', 'blocks6-task02', 'blocks6-task03', 'blocks6-task04', 'blocks6-task05', 'blocks6-task06', 'blocks6-task07', 'blocks6-task08', 'blocks6-task09', 'blocks6-task10', 'blocks7-task01', 'blocks7-task02', 'blocks7-task03', 'blocks7-task04', 'blocks7-task05',

In [138]:
results = np.load("tests/test_vectors/blocks.npz")
X = results['feature']
Y = results['label']
print(X,Y)
print(X.shape, Y.shape)

[[ 9.  6. 12. ...  4.  4.  0.]
 [ 6.  5. 10. ...  4.  3.  0.]
 [ 1.  0.  0. ...  2.  0.  0.]
 ...
 [ 2.  2.  3. ...  3.  2.  0.]
 [ 0.  1.  2. ...  2.  0.  0.]
 [ 3.  0.  0. ...  2.  0.  0.]] [[ 4.  0.]
 [ 3.  0.]
 [ 2.  0.]
 [ 6.  1.]
 [ 5.  1.]
 [ 4.  1.]
 [ 3.  1.]
 [ 2.  1.]
 [ 8.  2.]
 [ 7.  2.]
 [ 6.  2.]
 [ 5.  2.]
 [ 4.  2.]
 [ 3.  2.]
 [ 2.  2.]
 [ 6.  3.]
 [ 5.  3.]
 [ 4.  3.]
 [ 3.  3.]
 [ 2.  3.]
 [ 6.  4.]
 [ 5.  4.]
 [ 4.  4.]
 [ 3.  4.]
 [ 2.  4.]
 [ 6.  5.]
 [ 5.  5.]
 [ 4.  5.]
 [ 3.  5.]
 [ 2.  5.]
 [ 2.  6.]
 [ 4.  7.]
 [ 3.  7.]
 [ 2.  7.]
 [ 8.  8.]
 [ 7.  8.]
 [ 6.  8.]
 [ 5.  8.]
 [ 4.  8.]
 [ 3.  8.]
 [ 2.  8.]
 [ 6.  9.]
 [ 5.  9.]
 [ 4.  9.]
 [ 3.  9.]
 [ 2.  9.]
 [ 4. 10.]
 [ 3. 10.]
 [ 2. 10.]
 [ 4. 11.]
 [ 3. 11.]
 [ 2. 11.]
 [ 8. 12.]
 [ 7. 12.]
 [ 6. 12.]
 [ 5. 12.]
 [ 4. 12.]
 [ 3. 12.]
 [ 2. 12.]
 [ 6. 13.]
 [ 5. 13.]
 [ 4. 13.]
 [ 3. 13.]
 [ 2. 13.]
 [ 8. 14.]
 [ 7. 14.]
 [ 6. 14.]
 [ 5. 14.]
 [ 4. 14.]
 [ 3. 14.]
 [ 2. 14.]
 [ 8. 15.]
 [ 7. 15.]
 [ 6.

# SVM PART

In [8]:
import itertools
import numpy as np
import numpy.random as rd
from sklearn import svm, linear_model
from sklearn.model_selection import KFold
from scipy import stats
from svm import synthetic_data as sdata

In [9]:
np.random.seed(2)
# p_index, test_data, cost = sdata.create_synthetic_data(25, 7, 100, 5, 64, 23, 3)
p_index, test_data, cost = sdata.create_synthetic_data(5, 5, 100, 2, 10, 23, 3)
print(f"test_data has shape: {test_data.shape}\ncost has shape: {cost.shape}\n")
print("first vector in data set:\n",np.round(test_data[0]))
cost_display = cost[0:min(len(cost),7)]
print("\nfirst 7 element in cost vector:\n",cost_display)
print("the index for problems: ",p_index)

test_data has shape: (28, 55)
cost has shape: (28, 2)

first vector in data set:
 [23.  8. 17. 12. 20.  2.  3.  2.  1.  1.  0.  2.  1.  2.  1.  3.  3.  4.
  1.  2.  4.  1.  1.  1.  3.  3.  1.  2.  4.  3.  2.  2.  3.  3.  4.  2.
  4.  3.  1.  4.  2.  4.  2.  1.  4.  4.  1.  2.  2.  2.  2.  0.  1.  3.
  4.]

first 7 element in cost vector:
 [[ 8  0]
 [15  0]
 [ 8  1]
 [39  1]
 [58  1]
 [37  1]
 [34  1]]
the index for problems:  [ 0  2 11 18 20 28]


In [136]:
"""
implementation inspired from
https://gist.github.com/agramfort/2071994
"""

import itertools
import numpy as np

from sklearn import svm, linear_model
from sklearn.model_selection import KFold


def transform_pairwise(X, y):
    """
    Transforms data into pairs for convex relaxation of kendal rank correlation coef
    In this method, all pairs are choosen, except for those that have the same target value or equal cost
    Inputs
    ----------
    X : array, shape (n_samples, n_features)
        The input feature vec of states from of several problems
    y : array, shape (n_samples,) or (n_samples, 2)
        The input cost vector. If it's a 2D array, the second column represents
        the problem index
    Returns
    -------
    X_trans : array, shape (k, n_feaures)
        Difference between features of states (si - sj), only consider the state pair from the same problem
    y_trans : array, shape (k,)
        Output rank labels of values {-1, +1}, 1 represent si has potentially larger cost than sj (further away from goal)
    """
    X_new = []
    y_new = []
    if y.ndim == 1:
        y = np.c_[y, np.ones(y.shape[0])]
    comb = itertools.combinations(range(X.shape[0]), 2)
    for k, (i, j) in enumerate(comb):
        if y[i, 0] == y[j, 0] or y[i, 1] != y[j, 1]:
            # skip if they have the same cost or are from different problem group
            continue
        # otherwise, make the new pair-wise data
        X_new.append(X[i] - X[j])
        y_new.append(np.sign(y[i, 0] - y[j, 0])) # y = 1 if xi further away (larger cost), Vice Vesa
        # randomly output some negative values for training purpose
        if y_new[-1] != (-1) ** k:
            y_new[-1] = - y_new[-1]
            X_new[-1] = - X_new[-1]
    return np.asarray(X_new), np.asarray(y_new)


class RankSVM(svm.LinearSVC):
    """
    Performs pairwise ranking svm with an underlying LinearSVC model
    initialise with a C of regularization term
    default using hinge loss
    """
    
    def __init__(self, C = 1.0):
        super(RankSVM, self).__init__()
        self.C = C
        self.loss = 'hinge'
        
        
    def fit(self, X, y):
        """
        Fit a pairwise ranking model by first transfer it into pairwise than fitting
        Inputs
        ----------
        X : array, shape (n_samples, n_features)
        y : array, shape (n_samples,) or (n_samples, 2)
        Returns
        -------
        self
        """
        X_trans, y_trans = transform_pairwise(X, y)
        super(RankSVM, self).fit(X_trans, y_trans)
        return self

    def predict(self, X):
        """
        Predict an ordering on X. For a list of n samples, this method
        returns a list from 0 to n-1 with the relative order of the rows of X.
        Inputs
        ----------
        X : array, shape (n_samples, n_features)
        Returns
        -------
        rtn: array, shape (n_samples,)
            Returns a list of integers representing the relative order of
            the rows in X.
        """
        if hasattr(self, 'coef_'):
            return np.argsort(np.dot(X, self.coef_.T).flatten())
        else:
            raise ValueError("Must call fit() prior to predict()")

    def score(self, X, y):
        """
        Returns the accuracy for the rank prediction, from 0-1
        """
        X_trans, y_trans = transform_pairwise(X, y)
        return np.mean(super(RankSVM, self).predict(X_trans) == y_trans)


In [137]:
my_svm = RankSVM()

In [144]:
results = np.load("tests/test_vectors/blocks.npz")
X = results['feature']
Y = results['label']
# print(X,Y)
print(X.shape, Y.shape)


my_svm.fit(X, Y)

tx, ty = generate_training_data('domain/blocks/domain.pddl', 'domain/blocks/blocks/blocks3/task52.pddl', 'domain/blocks/plans/blocks3-task52_1800.out', 100)
my_svm.predict(tx)
my_svm.score(tx, ty)

(151, 39) (151, 2)


1.0

In [145]:
print(np.mean(np.array([1,0,1])))

0.6666666666666666


In [174]:
%load_ext autoreload
%autoreload 2

import numpy as np
import numpy.random as rd
from data import data_generator

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [176]:
dg.generate_feature_vec()

AttributeError: module 'data.data_generator' has no attribute 'generate_feature_vec'